In [1]:
import pandas as pd
import numpy as np
import torch

from matplotlib import pyplot as plt 

import time

In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchvision import datasets, models, transforms, utils
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T
import torchvision

import os
import copy

from skimage import io, transform

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   

In [3]:
from pydub import AudioSegment
from pydub.playback import play
import librosa

In [4]:
print(torchvision.__version__)

0.15.2+cu118


In [5]:
torch.cuda.is_available()

True

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
for model_dir in os.listdir("./"):
    if model_dir.split(".")[-1] != "pt":
        continue
    print(model_dir)   
        



best.pt
CNN_0.962963.pt
MLP_0.761364.pt
MLP_0.851064.pt
MLP_0.872340.pt
MLP_0.888889new_model3.pt
MLP_0.893617.pt
MLP_0.914894.pt
MLP_0.919753.pt
MLP_0.919753new_model2.pt
MLP_0.932099new_model3.pt
MLP_0.936170.pt
MLP_0.936170_best.pt
MLP_0.936170_old.pt
MLP_0.938272new_model3.pt


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
import pyaudio
import math
from scipy import signal
import soundfile as sf
import gradio as gr
import sounddevice as sd

In [10]:
class CNN_Audio_Model(torch.nn.Module):
    
    def __init__(self):
        super(CNN_Audio_Model, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding="same")
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.activation1 = torch.nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding="same")
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.activation2 = torch.nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.1)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding="same")
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.activation3 = torch.nn.ReLU()
        self.dropout3 = nn.Dropout(p=0.1)
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding="same")
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        self.bn4 = nn.BatchNorm2d(num_features=64)
        self.activation4 = torch.nn.ReLU()
        self.dropout4 = nn.Dropout(p=0.3)
        
        fc_dim_in = 5 * 8 * 64
        
        self.fc6 = nn.Linear(in_features=fc_dim_in, out_features=128)
        self.bn6 = nn.BatchNorm2d(num_features=128)
        self.activation6 = torch.nn.ReLU()
        self.dropout6 = nn.Dropout(p=0.5)
        
        
        self.fc7 = nn.Linear(in_features=128, out_features=4)
        self.bn7 = nn.BatchNorm2d(num_features=128)
        self.activation7 = torch.nn.ReLU()
        self.dropout7 = nn.Dropout(p=0.25)
        
        self.fc8 = nn.Linear(in_features=128, out_features=32)
        self.bn8 = nn.BatchNorm2d(num_features=32)
        self.activation8 = torch.nn.ReLU()
        self.dropout8 = nn.Dropout(p=0.25)  
        
        self.fc9 = nn.Linear(in_features=32, out_features=4)
        
        self.softmax = torch.nn.Softmax(dim = 1)
        
        

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation2(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.activation3(x)
        x = self.pool3(x)
        # x = self.dropout3(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.activation4(x)
        x = self.pool4(x)
        x = self.dropout4(x)
        
#         x = self.conv5(x)
#         x = self.bn5(x)
#         x = self.activation5(x)
#         x = self.pool5(x)
#         x = self.dropout5(x)
        
        # print(x.size())
        fc_dim_in = 5 * 8 * 64
        x = x.view(-1, fc_dim_in)
        # print(x.size())
        
        x = self.fc6(x)
        # x = self.bn6(x)
        x = self.activation6(x)
        x = self.dropout6(x)

        x = self.fc7(x)
        # x = self.bn7(x)
        # x = self.activation7(x)
        # x = self.dropout7(x)        

#         x = self.fc8(x)
#         x = self.bn8(x)
#         x = self.activation8(x)
#         x = self.dropout8(x)         

#         x = self.fc9(x)
        
        x = self.softmax(x)
        
        # return the output
        return x

CNN_model = CNN_Audio_Model()

print('The model:')
print(CNN_model)

# print('\n\nModel params:')
# for param in MLP_model.parameters():
#     print(param)

The model:
CNN_Audio_Model(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.1, inplace=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): ReLU()
  (dropout2): Dropout(p=0.1, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation3): ReLU()
  (dropout3): Dropout(p=0.1, inplace=False)
  (conv4): Conv2d(6

In [11]:
current_dir = os.getcwd()
model_path = current_dir + '\\' + "CNN_0.962963.pt"

test_model = CNN_Audio_Model()

test_model = torch.load(model_path)


# test_model = torch.load(model_path)

test_model.eval()
test_model = test_model.to(device)

## processor function for input data

In [12]:
SAMPLE_RATE = 16000

def proccess_cnn(frame):
        
        dir_dict = {0: "Down", 1: "Left", 2: "Right", 3: "Up"}
        
        tensor = frame
        

        print(tensor.size())
        
#         target_size = 48000
#         tensor = torch.nn.functional.pad(tensor, (0, target_size - tensor.size(1), 0, 0), 'constant', 0)      
        tensor = torch.nn.functional.pad(tensor, (1, 31999), mode='constant', value=0)
        
        print(tensor.size())

        tensor = librosa.feature.melspectrogram(y= tensor.numpy(), sr= SAMPLE_RATE, n_mels= 128)
        tensor = librosa.power_to_db(tensor, ref= np.max)
        tensor = torch.tensor(tensor, dtype=torch.float32)
        
#         tensor = nn.functional.normalize(tensor, dim=-1)

        print(tensor.size())

        tensor = torch.reshape(tensor, (1, 1, 128, 94))
        
        tensor = tensor.to(device)

        output = test_model(tensor)

        preds = output
        
        print(preds)
        

        predicted_value = torch.argmax(preds.to("cpu"), dim=1)

        # Return the output as a numpy array
        final_out = dir_dict[predicted_value.detach().numpy().item()]
        return final_out

## Helper function for gradio

####  Hint: in gradio type "start" and then submit to start recording

In [38]:
def transcribe_audio(text):
    if text == "start":
        input_format = pyaudio.paInt16
        input_channels = 1
        input_sample_rate = 16000  # Changed to 16000 samples per second
        chunk_size = 2048
        output_format = 'WAV'
        output_channels = 1
        output_sample_rate = 16000
        output_duration = 3  # seconds
        num_output_frames = output_sample_rate * output_duration

        # Create a PyAudio object
        audio = pyaudio.PyAudio()

        print('Recording...')

        # Open the microphone stream
        stream = audio.open(format=input_format, channels=input_channels,
                            rate=input_sample_rate, input=True, frames_per_buffer=chunk_size)

        # Start the stream
        since = time.time()
        stream.start_stream()

        # Record 3 seconds of audio
        recorded_frames = []
        for i in range(int(input_sample_rate / chunk_size * output_duration) - 15):
            data = stream.read(chunk_size)
            recorded_frames.append(data)

        # Stop the stream
        time_elapsed = time.time() - since
        print(f"duration: {time_elapsed}")
        stream.stop_stream()
        stream.close()

        print('Processing...')

        # Convert the recorded audio to a NumPy array
        recorded_data = np.frombuffer(b''.join(recorded_frames), dtype=np.int16)
        recorded_data = recorded_data[:16000]
        sd.play(recorded_data, 16000)
        recorded_data = recorded_data.astype(np.float16)
        print(recorded_data.shape)

        # Reshape the audio to (1, 48000) size
        audio_tensor = torch.from_numpy(recorded_data).reshape(1, -1)
        print(audio_tensor.size())
        # Process the audio with the pre-trained model
        with torch.no_grad():
            output = proccess_cnn(audio_tensor)

        return f'Predicted label: {output}'
    else:
        return "invalid input"

# define the inputs and outputs for the Gradio interface
audio_output = gr.outputs.Textbox(label="Output Label")

# create the Gradio interface
gr.Interface(fn=transcribe_audio, inputs="text", outputs=audio_output, title="Audio Classification").launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


Recording...
duration: 1.0310699939727783
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[9.4560e-01, 4.8155e-03, 4.9428e-02, 1.5732e-04]], device='cuda:0')
Recording...
duration: 1.0319890975952148
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[9.8919e-01, 9.9456e-04, 9.8091e-03, 3.6055e-06]], device='cuda:0')
Recording...
duration: 1.0321147441864014
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[1.2736e-01, 8.7122e-01, 1.4286e-03, 4.7970e-09]], device='cuda:0')
Recording...
duration: 1.0313353538513184
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[2.5788e-03, 1.8502e-02, 9.7881e-01, 1.1439e-04]], device='cuda:0')
Recording...
duration: 1.0310730934143066
Processing...
(160

In [13]:
import threading
import time


In [14]:
import pygame
import random
import os
import pyaudio
thread_run = 0
input_format = pyaudio.paInt16
input_channels = 1
input_sample_rate = 16000  # Changed to 16000 samples per second
chunk_size = 2048
output_format = 'WAV'
output_channels = 1
output_sample_rate = 16000
output_duration = 3  # seconds
num_output_frames = output_sample_rate * output_duration

# Create a PyAudio object
audio = pyaudio.PyAudio()



# Initialize Pygame
pygame.init()

# Set the screen dimensions
SCREEN_WIDTH = 500
SCREEN_HEIGHT = 600

# Set the colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Set the font
FONT = pygame.font.Font(None, 36)

# Set the frame rate
FPS = 60

# Set the number of lives
LIVES = 3

# Set the object spawn rate
SPAWN_RATE = 5000  # milliseconds

# Set the explosion duration
EXPLOSION_DURATION = 50  # milliseconds

# Set the score value
SCORE_VALUE = 10

# Load the background image
background_image = pygame.image.load(".\\game\\game\\sea.png")

# Load the airplane image
airplane_image = pygame.image.load(".\\game\\game\\airplane3.png")

# Load the object image
object_image = pygame.image.load(".\\game\\game\\bomb_sharif.png")

# Load the explosion image
explosion_image = pygame.image.load(".\\game\\game\\explosion.png")

# Set the game window
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Airplane Game")

# Create a clock to control the frame rate
clock = pygame.time.Clock()

# Define the airplane class
class Airplane(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = airplane_image
        self.rect = self.image.get_rect()
        self.rect.centerx = SCREEN_WIDTH // 2
        self.rect.bottom = SCREEN_HEIGHT - 10
        self.speed_x = 0
        self.speed_y = 0

    def update(self):
        # Move the airplane based on user input
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y

        # Keep the airplane inside the screen
        if self.rect.left < 0:
            self.rect.left = 0
        elif self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top < 0:
            self.rect.top = 0
        elif self.rect.bottom > SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

    def shoot(self):
        # Create a bullet sprite and add it to the appropriate groups
        bullet = Bullet(self.rect.centerx, self.rect.top)
        all_sprites.add(bullet)
        bullets.add(bullet)

# Define the object class
class Object(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = object_image
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(2*self.rect.width, SCREEN_WIDTH - 2*self.rect.width)
        self.rect.y = -self.rect.height
        self.speed_y = 10

    def update(self):
        # Move the object down the screen
        self.rect.y += self.speed_y

        # Check if the object has collided with the airplane
        if pygame.sprite.collide_rect(self, airplane):
            # Create an explosion sprite and add it to the appropriate groups
            explosion = Explosion(self.rect.center)
            all_sprites.add(explosion)
            explosions.add(explosion)

            # Remove the object sprite
            self.kill()

            # Decrease the number of lives
            global LIVES
            LIVES -= 1

            # End the game if there are no lives remaining
            if LIVES == 0:
                end_game()

        # Check if the object has gone off the bottom of the screen
        elif self.rect.top > SCREEN_HEIGHT:
            # Remove the object sprite
            self.kill()

# Define the bullet class
class Bullet(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((3, 15))
        self.image.fill(RED)
        self.rect = self.image.get_rect()
        self.rect.centerx = x
        self.rect.bottom = y
        self.speed_y = -50

    def update(self):
        # Move the bullet up the screen
        self.rect.y += self.speed_y

        # Remove the bullet sprite if it goes off the top of the screen
        if self.rect.bottom < 0:
            self.kill()

# Define the explosion class
class Explosion(pygame.sprite.Sprite):
    def __init__(self, center):
        super().__init__()
        self.image = explosion_image
        self.rect = self.image.get_rect()
        self.rect.center = center
        self.duration = EXPLOSION_DURATION

    def update(self):
        # Decrease the duration of the explosion
        self.duration -= clock.tick(FPS)

        # Remove the explosion sprite if the duration is zero
        if self.duration <= 0:
            self.kill()

# Define the function to spawn objects
def spawn_object():
    # Create an object sprite and add it to the appropriate groups
    obj = Object()
    all_sprites.add(obj)
    objects.add(obj)

# Define the function to end the game
def end_game():
    # Create the result message
    result_msg = "Dr Sharifian killed us! Your score was: {}".format(score)

    # Display the result message
    result_text = FONT.render(result_msg, True, WHITE)
    result_rect = result_text.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
    screen.blit(result_text, result_rect)
    pygame.display.flip()

    # Wait for 3 seconds before quitting the game
    pygame.time.delay(3000)
    pygame.quit()
    quit()

# Create the airplane sprite and add it to the appropriate groups
airplane = Airplane()
all_sprites = pygame.sprite.Group()
all_sprites.add(airplane)

# Create the bullet and object groups
bullets = pygame.sprite.Group()
objects = pygame.sprite.Group()

# Create the explosion group
explosions = pygame.sprite.Group()

# Set the initial score and time
score = 0
time = pygame.time.get_ticks()

# Start the game loop
running = True


while running:
    # Handle events

    # Open the microphone stream
    print('Recording...')
    stream = audio.open(format=input_format, channels=input_channels, rate=input_sample_rate, input=True, frames_per_buffer=chunk_size)

    # Start the stream
#     since = time.time()
    stream.start_stream()

    # Record 3 seconds of audio
    recorded_frames = []
    for i in range(int(input_sample_rate / chunk_size * output_duration) - 5):
        data = stream.read(chunk_size)
        recorded_frames.append(data)

    # Stop the stream
#     time_elapsed = time.time() - since
#     print(f"duration: {time_elapsed}")
    stream.stop_stream()
    stream.close()

    print('Processing...')

    # Convert the recorded audio to a NumPy array
    recorded_data = np.frombuffer(b''.join(recorded_frames), dtype=np.int16)
    recorded_data = recorded_data[:16000]
    recorded_data = recorded_data.astype(np.float16)
    print(recorded_data.shape)
    
    # Reshape the audio to (1, 48000) size
    audio_tensor = torch.from_numpy(recorded_data).reshape(1, -1)
    print(audio_tensor.size())
    
    # Process the audio with the pre-trained model
    with torch.no_grad():
        output = proccess_cnn(audio_tensor)

    # Print the predicted label
    
    print(f'Predicted label: {output}')



    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
        #     if event.key == pygame.K_LEFT:
        #         airplane.speed_x = -10
        #     elif event.key == pygame.K_RIGHT:
        #         airplane.speed_x = 10
        #     elif event.key == pygame.K_UP:
        #         airplane.speed_y = -10
        #     elif event.key == pygame.K_DOWN:
        #         airplane.speed_y = 10
            if event.key == pygame.K_SPACE:
                airplane.shoot()
        # elif event.type == pygame.KEYUP:
        #     if event.key == pygame.K_LEFT and airplane.speed_x < 0:
        #         airplane.speed_x = 0
        #     elif event.key == pygame.K_RIGHT and airplane.speed_x > 0:
        #         airplane.speed_x = 0
        #     elif event.key == pygame.K_UP and airplane.speed_y < 0:
        #         airplane.speed_y = 0
        #     elif event.key == pygame.K_DOWN and airplane.speed_y > 0:
        #         airplane.speed_y = 0
    if output == "Left":
        airplane.speed_x = -50
        airplane.speed_y = 0
    elif output == "Right":
        airplane.speed_x = 50
        airplane.speed_y = 0
    elif output == "Up":
        airplane.speed_y = -50
        airplane.speed_x = 0 
    elif output == "Down":
        airplane.speed_x = 0
        airplane.speed_y = 50
    # Spawn objects every 5 seconds
    if pygame.time.get_ticks() - time >= SPAWN_RATE:
        spawn_object()
        time = pygame.time.get_ticks()

    # Update the sprites
    all_sprites.update()

    # Check for collisions between bullets and objects
    hits = pygame.sprite.groupcollide(objects, bullets, True, True)
    for hit in hits:
        # Create an explosion sprite and add it to the appropriate groups
        explosion = Explosion(hit.rect.center)
        all_sprites.add(explosion)
        explosions.add(explosion)

        # Increase the score
        score += SCORE_VALUE

    # Draw the background and sprites
    screen.blit(background_image, (0, 0))
    all_sprites.draw(screen)

    # Draw the score and lives
    score_text = FONT.render("Score: {}".format(score), True, WHITE)
    score_rect = score_text.get_rect(topright=(SCREEN_WIDTH - 10, 10))
    screen.blit(score_text, score_rect)

    lives_text = FONT.render("Lives: {}".format(LIVES), True, WHITE)
    lives_rect = lives_text.get_rect(topleft=(10, 10))
    screen.blit(lives_text, lives_rect)

    # Update the display
    pygame.display.flip()

    # End the game if there are no lives remaining
    if LIVES == 0:
        end_game()

    # Control the frame rate
    clock.tick(FPS)

# Quit the game
pygame.quit()
quit()

pygame 2.5.0 (SDL 2.28.0, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Recording...
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[0.0010, 0.0791, 0.9169, 0.0030]], device='cuda:0')
Predicted label: Right
Recording...
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[9.2693e-08, 6.1461e-01, 3.8538e-01, 3.2261e-06]], device='cuda:0')
Predicted label: Left
Recording...
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[7.1141e-09, 4.0158e-01, 5.9842e-01, 2.3752e-06]], device='cuda:0')
Predicted label: Right
Recording...
Processing...
(16000,)
torch.Size([1, 16000])
torch.Size([1, 16000])
torch.Size([1, 48000])
torch.Size([1, 128, 94])
tensor([[6.5598e-07, 6.4793e-01, 3.5206e-01, 3.8826e-06]], device='cuda:0')
Predicted 